# **Municipality Insights: Navigating Alberta's Multidimensional Data**

![title](albertalogo.png)

Harjot Dhaliwal (30051859)

Lukas Escoda (30211208)

Shabbir Khandwala (30219011)

Gurdeep Panag (30101520)

University of Calgary

DATA 604: Working with Data at Scale

Instructor: Dr. Ovens

# **Introduction**

Embarking on this comprehensive project, our primary goal is to delve into Alberta's open data, extracting insights that hold significance for both established residents and newcomers. Our exploration encompasses three key sectors: Housing, Health/Environment, and Business/Economy. Recognizing the importance of a data-centric approach, we are committed to empowering informed decision-making, catalyzing positive change, and shedding light on various aspects of Alberta.

This initiative serves to enable current and potential Albertan citizens and promote a stronger collective understanding of its municipalities. After establishing a common ground of Alberta's recent events (i.e. housing crisis (Calgary City Council to Hold Special Meeting to Address ‘Housing Crisis'., (2023).), transportation projects (Alberta NDP calls on Province to Fully Commit to Calgary's Green Line Funding., (2023).), and wildfires (Wildfires in Alberta Could Burn All Winter: Official., (2023).)), we gained a newfound interest in gaining a better understanding of these sectors relating to Alberta specifically over the past decade. The diverse perspectives within our team, including both newcomers and long-term residents, add depth to our understanding of Alberta's nuances.

In the housing sector, our research seeks answers to compelling questions, unraveling the unique characteristics of Alberta's priciest housing municipalities, tracking the evolution of rent rates over time, and identifying areas with the highest concentration of family-style homes.

Shifting focus to Health/Environment, our inquiries delve into how municipalities compare in terms of air quality and population growth, aiming to paint a historical picture of the environmental landscape and demographic trends in Alberta.

In the realm of Business/Economy, our research aims to untangle the intricate relationship between taxes and business creation. Additionally, we seek to reveal the impact of government project funding on the proliferation of businesses within the province. By addressing these critical questions, our project aims to offer nuanced insights that can guide strategic decisions and contribute to the sustainable development of Alberta.

# **Individual Datasets**

We've chosen datasets by exploring the Government of Alberta's open data portal, for CSV files with matching primary keys (municipality and year). After sorting these datasets into three categories – Housing, Health/Environment, and Business/Economy – we proceeded with data analysis, in accordance with the Alberta Open Government License.

Working with these datasets has been a journey of discovery and skill improvement. Spending hands-on time with SQL and within our shared database was productive in learning how to clean, organize, and combine our raw datasets into meaningful tables. This process highlights the challenges of inconsistent data, like missing values or info placed in the wrong columns. We've noticed patterns in how often certain data was collected, the lack thereof in data collection for smaller municipalities, and the sheer amount of government data that had been compiled for certain datasets (i.e. population census). We carefully filter rows to best answer our guiding questions and joined relevant datasets by municipality and year.

In the realm of the housing industry, our discerning eye fell upon datasets such as Average Rent by Municipality, Percent Single Family Houses by Municipality, and Property Assessments by Municipality. These choices were motivated by a strategic focus on factors deemed vital to newcomers in Alberta. Understanding the trends in these metrics over time is pivotal for those unable to outright purchase a home, aiding them in discerning affordability across municipalities. Moreover, insights into the percentage of single-family homes cater to the diverse needs of moving families, contrasting the preferences of families with young children against those of single professionals. 

Transitioning to the health and environment domain, our chosen datasets encompassed Air Quality by Municipality, Births and Deaths by Municipality, and Population by Municipality. Ensuring that residents and newcomers are aware of trends in air quality, especially given the recent severity of wildfires that have adversely affected the health of many Albertans, will be incredibly informative for those seeking a data-driven balanced perspective of what has really happened over time. Delving into population dynamics, our datasets provided the foundation for calculating the birth rate per 100 people as well as the domestic net growth for each municipality, to illustrate how the growth rates of municipalities have changed over time. 

In the business and economy sphere, our scrutiny extended to Tax Rates by Municipality, Business Incorporations by Municipality, Number of Businesses by Municipality, and Major Projects by Municipality. The emphasis here was on unraveling the economic fabric of each municipality. Tax rates and business incorporations were deemed pivotal, acting as catalysts for creating opportunities and jobs, thereby stimulating local economies. The dataset on major projects further enriched our insights, offering a window into the commitment of each municipality to growth and community betterment, factors highly influential for those contemplating a move.

In case needed, our database credentials are:

Username: l01-13

Password: rmULBbdZSEvj0

Database: l01-13

Importing Packages and Connecting to the Database:

In [19]:
import numpy as np
import pandas as pd
# suppresses scientific notation in pandas dataframes
pd.options.display.float_format = '{:.0f}'.format
import geopandas as gp

import sqlalchemy as sq
import mysql.connector
from mysql.connector import errorcode

import os

# ignores warnings
import warnings
warnings.filterwarnings("ignore")

In [20]:
# accessing password from stored file
with open("password.txt") as f:
    passw = f.read()

# creating the connection object for connecting the mysql python connector the database
connection = mysql.connector.connect(user = "l01-13", 
                                       password = passw,
                                       host = "datasciencedb2.ucalgary.ca", 
                                       database = "l01-13",
                                       allow_local_infile = True)

# creating the engine object which would be connecting to the database for sqlalchemy
engine = sq.create_engine("mysql+mysqlconnector://l01-13:{}@datasciencedb2.ucalgary.ca/l01-13".format(passw))

In [3]:
# iterating over the raw datasets present in the dataset folder and loading them into the database as tables
# do not run again
for (root, _, files) in os.walk(os.path.join(os.getcwd(), "Datasets")):
    for file in files:
        pd.read_csv(os.path.join(root, file)).to_sql(name = "stg_" + file.lower().split(".")[0].replace(" ", "_"), con = engine, if_exists = "replace", index=False)
        print(f"Successfully loaded: {file}")

Successfully loaded: Air Quality.csv
Successfully loaded: Average Rent.csv
Successfully loaded: Births Deaths.csv
Successfully loaded: Dwelling Units.csv
Successfully loaded: Employment Census.csv
Successfully loaded: Incorporations.csv
Successfully loaded: Major Projects.csv
Successfully loaded: Median Income.csv
Successfully loaded: Number of Businesses.csv
Successfully loaded: Population.csv
Successfully loaded: Property Assessments.csv
Successfully loaded: Single Family Houses.csv
Successfully loaded: Tax Rates.csv


### Data Cleaning/Wrangling

Cleaning/Wrangling Health/Environment Tables:

In [21]:
health_cleaning_queries = [
# dropping the table to create in case it exists to ensure code remains runnable
"""
DROP TABLE IF EXISTS int_births_deaths;
""",
# creating the births and deaths table
"""
CREATE TABLE int_births_deaths AS
SELECT CSDUID AS community_id, TRIM(CSD) AS community_name, CAST(Period AS INT) AS year,
CAST(SUM(IF(IndicatorSummaryDescription = "Births", OriginalValue, 0)) AS INT) AS total_births,
CAST(SUM(IF(IndicatorSummaryDescription = "Deaths", OriginalValue, 0)) AS INT) AS total_deaths
FROM stg_births_deaths WHERE CSDUID != "4800000"
GROUP BY CSDUID, CSD, Period;
""",
# dropping the table to create in case it exists to ensure code remains runnable
"""
DROP TABLE IF EXISTS int_air_quality;
""",
# creating the air quality table
"""
CREATE TABLE int_air_quality AS
SELECT CSDUID AS community_id, 
TRIM(CSD) AS community_name, 
CAST(Period AS INT) AS year, 
ROUND(SUM(OriginalValue)*365,0) AS Days,
CASE
WHEN CAST(REPLACE(`Air Quality Health Index`, "+", "") AS INT) BETWEEN 1 AND 3 THEN "High"
WHEN CAST(REPLACE(`Air Quality Health Index`, "+", "") AS INT) BETWEEN 4 AND 6 THEN "Moderate"
ELSE "Low" 
END AS quality
from stg_air_quality 
WHERE CSDUID != "4800000"
GROUP BY CSDUID, CSD, period,
CASE
WHEN CAST(REPLACE(`Air Quality Health Index`, "+", "") AS INT) BETWEEN 1 AND 3 THEN "High"
WHEN CAST(REPLACE(`Air Quality Health Index`, "+", "") AS INT) BETWEEN 4 AND 6 THEN "Moderate"
ELSE "Low" 
END;
"""]

# looping through each query above and executing it in the database
for query in health_cleaning_queries:
    # creating the cursor object
    cursor = connection.cursor()
    # executing the query
    cursor.execute(query)
    # closing the cursor object
    cursor.close()

# committing the queries to the database
connection.commit()

Cleaning/Wrangling Housing Tables:

In [22]:
housing_cleaning_queries = [
# dropping the table to create in case it exists to ensure code remains runnable
"""
DROP TABLE IF EXISTS int_property_assessments;
""",
# creating the property assessments table
"""
CREATE TABLE int_property_assessments AS
SELECT CSDUID AS community_id, TRIM(CSD) AS community_name, CAST(period AS INT) AS year, OriginalValue AS assessed_value
from stg_property_assessments WHERE CSDUID != "4800000" AND UPPER(TRIM(`Property Type`)) = "Residential";
""",
# dropping the table to create in case it exists to ensure code remains runnable
"""
DROP TABLE IF EXISTS int_single_family_houses;
""",
# creating the table for % single family houses
"""
CREATE TABLE int_single_family_houses AS
SELECT a.CSDUID AS community_id, TRIM(a.CSD) AS community_name, CAST(a.period AS INT) AS year, 
ROUND(a.OriginalValue * 100, 2) AS dwelling_percentage,
FLOOR(a.OriginalValue * b.OriginalValue) AS dwelling_cnt
FROM stg_single_family_houses a
INNER JOIN stg_dwelling_units b ON a.CSDUID = b.CSDUID AND a.period = b.period
WHERE a.CSDUID != "4800000";
""",
# dropping the table to create in case it exists to ensure code remains runnable
"""
DROP TABLE IF EXISTS int_average_rent;
""",
# creating the table for average rent
"""
CREATE TABLE int_average_rent AS
SELECT CSDUID AS community_id, TRIM(CSD) AS community_name, CAST(period AS INT) AS year,
SUM(IF(UPPER(TRIM(`Rental Unit Type`)) = "3 - BEDROOM", OriginalValue, 0)) AS rental_rate_three_bedroom,
SUM(IF(UPPER(TRIM(`Rental Unit Type`)) = "2 - BEDROOM", OriginalValue, 0)) AS rental_rate_two_bedroom,
SUM(IF(UPPER(TRIM(`Rental Unit Type`)) = "1 - BEDROOM", OriginalValue, 0)) AS rental_rate_one_bedroom,
SUM(IF(UPPER(TRIM(`Rental Unit Type`)) = "BACHELOR", OriginalValue, 0)) AS rental_rate_bachelor
FROM stg_average_rent WHERE CSDUID != "4800000"
GROUP BY CSDUID, CSD, period;
"""]

# looping through each query above and executing it in the database
for query in housing_cleaning_queries:
    # creating the cursor object
    cursor = connection.cursor()
    # executing the query
    cursor.execute(query)
    # closing the cursor object
    cursor.close()

# committing the queries to the database
connection.commit()

Cleaning/Wrangling Business/Economy Tables:

In [23]:
business_cleaning_queries = [
# dropping the table to create in case it exists to ensure code remains runnable
"""
DROP TABLE IF EXISTS int_major_projects;
""",
# creating the major projects table
"""
CREATE TABLE int_major_projects AS
SELECT CSDUID AS community_id, TRIM(CSD) AS community_name, CAST(period AS INT) AS year,
sector, OriginalValue AS project_cost 
from stg_major_projects WHERE CSDUID != "4800000";
""",
# dropping the table to create in case it exists to ensure code remains runnable
"""
DROP TABLE IF EXISTS int_number_of_businesses;
""",
# creating the number of businesses table
"""
CREATE TABLE int_number_of_businesses AS
SELECT CSDUID AS community_id, TRIM(CSD) AS community_name, CAST(period AS INT) AS year,
SUBSTRING_INDEX(size, " ", 1) AS size, industry, NAICS AS industry_code, OriginalValue AS num_business 
from stg_number_of_businesses  WHERE CSDUID != "4800000";
""",
# dropping the table to create in case it exists to ensure code remains runnable
"""
DROP TABLE IF EXISTS int_median_income;
""",
# creating the median income table
"""
CREATE TABLE int_median_income
SELECT CSDUID AS community_id, TRIM(CSD) AS community_name, CAST(period AS INT) AS year,
SUM(IF(IndicatorSummaryDescription LIKE "All%", OriginalValue, 0)) AS overall_income,
SUM(IF(IndicatorSummaryDescription LIKE "Couple%", OriginalValue, 0)) AS couple_income,
SUM(IF(IndicatorSummaryDescription LIKE "Lone%", OriginalValue, 0)) AS lone_income
FROM stg_median_income WHERE CSDUID != "4800000"
GROUP BY CSDUID, CSD, period;
""",
# dropping the table to create in case it exists to ensure code remains runnable
"""
DROP TABLE IF EXISTS int_population;
""",
# creating the population size table
"""
CREATE TABLE int_population AS
SELECT CSDUID AS community_id, TRIM(CSD) AS community_name, CAST(period AS INT) AS year,
SUM(IF(Gender = "Female" AND CAST(REPLACE(age, "+", "") AS INT) >= 15, OriginalValue, 0)) AS employable_female_count,
SUM(IF(Gender = "Male" AND CAST(REPLACE(age, "+", "") AS INT) >= 15, OriginalValue, 0)) AS employable_male_count,
SUM(IF(CAST(REPLACE(age, "+", "") AS INT) >= 15, OriginalValue, 0)) AS employable_total_count,
SUM(IF(Gender = "Female", OriginalValue, 0)) AS female_count,
SUM(IF(Gender = "Male", OriginalValue, 0)) AS male_count,
SUM(OriginalValue) AS total_count
FROM stg_population WHERE CSDUID != "4800000"
GROUP BY CSDUID, CSD, period;
""",
# dropping the table to create in case it exists to ensure code remains runnable
"""
DROP TABLE IF EXISTS int_employment_census;
""",
# creating the employment census table
"""
CREATE TABLE int_employment_census AS
SELECT CSDUID AS community_id, TRIM(CSD) AS community_name, CAST(period AS INT) AS year,
SUM(IF(IndicatorSummaryDescription LIKE "Participation%" AND Gender = "Male", OriginalValue, 0)) AS male_participation_rate,
SUM(IF(IndicatorSummaryDescription LIKE "Employment%" AND Gender = "Male", OriginalValue, 0)) AS male_employment_rate,
SUM(IF(IndicatorSummaryDescription LIKE "Unemployment%" AND Gender = "Male", OriginalValue, 0)) AS male_unemployment_rate,
SUM(IF(IndicatorSummaryDescription LIKE "Participation%" AND Gender = "Female", OriginalValue, 0)) AS female_participation_rate,
SUM(IF(IndicatorSummaryDescription LIKE "Employment%" AND Gender = "Female", OriginalValue, 0)) AS female_employment_rate,
SUM(IF(IndicatorSummaryDescription LIKE "Unemployment%" AND Gender = "Female", OriginalValue, 0)) AS female_unemployment_rate,
SUM(IF(IndicatorSummaryDescription LIKE "Participation%" AND Gender = "Both", OriginalValue, 0)) AS overall_participation_rate,
SUM(IF(IndicatorSummaryDescription LIKE "Employment%" AND Gender = "Both", OriginalValue, 0)) AS overall_employment_rate,
SUM(IF(IndicatorSummaryDescription LIKE "Unemployment%" AND Gender = "Both", OriginalValue, 0)) AS overall_unemployment_rate
FROM stg_employment_census WHERE CSDUID != "4800000"
GROUP BY CSDUID, CSD, period;
""",
# dropping the table to create in case it exists to ensure code remains runnable
"""
DROP TABLE IF EXISTS int_tax_rates;
""",
# creating the tax rates table
"""
CREATE TABLE int_tax_rates AS
SELECT CSDUID AS community_id, TRIM(CSD) AS community_name, CAST(period AS INT) AS year, OriginalValue AS tax_rate
FROM stg_tax_rates WHERE CSDUID != "4800000";
""",
# dropping the table to create in case it exists to ensure code remains runnable
"""
DROP TABLE IF EXISTS int_incorporations;
""",
# creating the number of incorporations table
"""
CREATE TABLE int_incorporations AS
SELECT CSDUID AS community_id, TRIM(CSD) AS community_name, CAST(period AS INT) AS year, OriginalValue AS num_incorporations
FROM stg_incorporations WHERE CSDUID != "4800000"                             
"""]

# looping through each query above and executing it in the database
for query in business_cleaning_queries:
    # creating the cursor object
    cursor = connection.cursor()
    # executing the query
    cursor.execute(query)
    # closing the cursor object
    cursor.close()

# committing the queries to the database
connection.commit()

# **Data Exploration**

### Health/Environment Queries

How do municipalities compare against each other in air quality?

Understanding and comparing Alberta's municipalities in air quality is vital for pinpointing areas needing intervention, safeguarding public health, and evaluating the success of local environmental efforts. This comparison sheds light on the impacts of natural events like wildfires and region-specific activities such as drilling or oil operations, providing crucial data for targeted responses and sustainable decision-making. Additionally, it offers residents insights into which areas offer better air quality, empowering individuals to make informed choices about where to live and contributing to improved overall well-being and health outcomes.

In [25]:
health_queries = [
# dropping the table to create in case it exists to ensure code remains runnable
"""
DROP TABLE IF EXISTS final_air_quality;
""",
# creating the final air quality table for the queries
"""
CREATE TABLE final_air_quality AS
SELECT community_name, year, Days AS days, quality
FROM int_air_quality
ORDER BY community_name, year, quality;
""",
# dropping the table to create in case it exists to ensure code remains runnable
"""
DROP TABLE IF EXISTS final_population_growth;
""",
# creating the final table for population growth queries
"""
CREATE TABLE final_population_growth AS 
SELECT 
bd.community_id, 
bd.community_name, 
bd.year, 
bd.total_births, 
bd.total_deaths, 
p.total_count, 
CAST((bd.total_births / p.total_count) * 100 AS DECIMAL(10,2)) AS births_per_100,
CAST((bd.total_births - bd.total_deaths) AS INT) AS net_growth
FROM `int_births_deaths` bd
INNER JOIN `int_population` p ON bd.community_id = p.community_id AND bd.year = p.year;
"""]

# looping through each query above and executing it in the database
for query in health_queries:
    # creating the cursor object
    cursor = connection.cursor()
    # executing the query
    cursor.execute(query)
    # closing the cursor object
    cursor.close()

# committing the queries to the database
connection.commit()

In [28]:
# displaying the newly created table for the query
display(pd.read_sql(""" 
SELECT *
FROM final_air_quality;
""", engine).head())

,community_name,year,days,quality
0,Airdrie,2017,340,High
1,Airdrie,2017,2,Low
2,Airdrie,2017,24,Moderate
3,Airdrie,2018,314,High
4,Airdrie,2018,5,Low


We have creating an interactive Tableau dashboard for this query, the link is:

https://public.tableau.com/app/profile/gurdeep.panag/viz/HealthEnvironmentDashboard/AirQualityDashboard?publish=yes

Our interactive air quality dashboard reveals no specific municipality consistently experiencing the highest number of days with low air quality (Air Quality Index 7, 8, 9, or 10+), signifying elevated levels of pollutants like ground-level ozone, particulate matter, and nitrogen dioxide. Such days are infrequent, often limited to just a few (typically < 7) throughout the year, mostly associated with localized natural disasters, such as wildfires. In 2022, Canmore reported the most significant anomaly, totaling 14 days of low air quality (non-consecutive, cumulative hours). In terms of moderate air quality, Edmonton and Calgary emerge as leaders, often competing, yet Edmonton typically surpasses Calgary by a week or two annually, with both cities ranging between 20-40 days per year. Other observed municipalities register significantly fewer days, likely due to heightened air pollutants in metropolitan areas. Fortunately, high air-quality days (Air Quality Index 1, 2, or 3), indicating safe air for the public, dominate across all Alberta municipalities for the majority of the year—a positive trend. The key takeaway is that Alberta generally enjoys high air quality throughout the year, except during natural disasters when widespread smog and reduced visibility affect all municipalities. For those prioritizing maximum air quality, choosing municipalities outside Edmonton and Calgary could minimize exposure to moderate air quality days.

How do municipalities compare against each other in population growth?

Understanding how Alberta's municipalities compare in terms of population growth is crucial for several reasons. It provides insights into demographic shifts, helping allocate resources efficiently based on varying needs in different areas. Comparing births per capita, total births, and net growth allows for targeted planning, such as healthcare infrastructure development and educational facilities, ensuring communities can support their evolving populations. Additionally, this data aids in formulating policies that address challenges related to aging populations or areas experiencing rapid growth, contributing to more balanced and sustainable socio-economic development across the province. Moreover, this knowledge offers residents valuable insights into their communities, empowering them to anticipate changes, plan for future needs, and actively engage in local decision-making processes. It enables individuals to advocate for resources and services that align with the evolving demographics of their areas, fostering a sense of ownership and contributing to the overall well-being and development of their communities.

In [29]:
# displaying the newly created table for the query
display(pd.read_sql(""" 
SELECT *
FROM final_population_growth;
""", engine).head())

,community_id,community_name,year,total_births,total_deaths,total_count,births_per_100,net_growth
0,4801003,Cypress County,2014,65,3,7556,1,62
1,4801003,Cypress County,2015,67,23,7826,1,44
2,4801003,Cypress County,2016,60,21,7849,1,39
3,4801003,Cypress County,2017,76,15,7968,1,61
4,4801003,Cypress County,2018,61,15,7937,1,46


We have creating an interactive Tableau dashboard for this query, the link is:

https://public.tableau.com/app/profile/gurdeep.panag/viz/HealthEnvironmentDashboard/PopulationDashboard?publish=yes

Our interactive population growth dashboard doesn’t reveal significant differences in births per 100 people across Alberta’s municipalities in the observed years. Instances of notably higher births per capita, highlighted on the choropleth map, mainly occurred in municipalities with extremely small populations (a couple hundred or less), skewing the birth rate per capita, suggesting higher fertility rates than the actual scenario. Moreover, the visualization of total births across municipalities provides a clearer picture of rapid growth areas, consistently showing Edmonton and Calgary leading in total births, reflecting their densely populated nature. Calgary typically records a few thousand more births than Edmonton. Additionally, our net growth visualization, representing births minus deaths, exposes several rural municipalities experiencing stagnant or negative growth, signifying local population decline (without accounting for immigration). Edmonton and Calgary consistently lead in net population growth year over year. In essence, if seeking a booming metropolis offering extensive amenities, resources, and government support, Calgary and Edmonton emerge as prime choices. These cities are expected to be focal points for provincial initiatives and funding, providing abundant opportunities for newcomers, families, and individuals seeking a dynamic environment.

### Housing Queries

Which 5 municipalities had the highest assessed property values in 2021, and what have been the trends of the average rent rates of these since 2017?

This would give us more information on which municipalities to look further into to see if they have higher than average rent rates, due to the average property assessment value being high, therefore we can determine if there is a relationship here between those variables and if higher property prices leads to and increase in rent rates, and how these values have changed over recent time.

This query will give us the top 10 municipalities with the highest property assessments in 2021.

In [30]:
# displaying the newly created table for the query
display(pd.read_sql(""" 
SELECT *
FROM int_property_assessments
WHERE year = 2021
ORDER BY assessed_value DESC
LIMIT 10;
""", engine))

,community_id,community_name,year,assessed_value
0,4806016,Calgary,2021,210609514058
1,4811061,Edmonton,2021,130776923655
2,4811052,Strathcona County,2021,17374566379
3,4806014,Rocky View County,2021,13434017078
4,4808011,Red Deer,2021,11643244387
5,4802012,Lethbridge,2021,10818621459
6,4811062,St. Albert,2021,10530843076
7,4806021,Airdrie,2021,9577785459
8,4801006,Medicine Hat,2021,7097441136
9,4819012,Grande Prairie,2021,7096111413


We would not like to choose Strathcona County, Rocky View County, St. Albert, or Airdrie, as these, at this point are just extensions of the larger cities of Edmonton and Calgary. We can now use this information in order to get a five year timeline of values from 2017 to 2021 for average property assessment value and average rent rates for the 5 municipalities of; Calgary, Edmonton, Red Deer, Lethbridge, and Medicine Hat.

In [34]:
housing_queries = [
# dropping the table to create in case it exists to ensure code remains runnable
"""
DROP TABLE IF EXISTS final_average_rent
""",
# creating the final table for 2021 property assessments and average rent rates
"""
CREATE TABLE final_average_rent AS
SELECT ar.community_name, ar.year, ar.rental_rate_three_bedroom, pa.assessed_value
FROM int_property_assessments pa
    RIGHT OUTER JOIN int_average_rent ar
        ON pa.community_name = ar.community_name AND pa.year = ar.year
WHERE (ar.community_name = 'Calgary' OR ar.community_name = 'Edmonton' OR ar.community_name = 'Lethbridge' OR
    ar.community_name = 'Medicine Hat' OR ar.community_name = 'Red Deer')
    AND (ar.year = 2017 OR ar.year = 2018 OR ar.year = 2019 OR ar.year = 2020 OR ar.year = 2021)
;
""",
# dropping the table to create in case it exists to ensure code remains runnable
"""
DROP TABLE IF EXISTS housing_query2;
""",
# creating the table for % of single family homes
"""
CREATE TABLE final_perc_single_family AS
SELECT community_name, year, dwelling_percentage
FROM int_single_family_houses
WHERE year = 2021
ORDER BY dwelling_percentage DESC;
"""]

# looping through each query above and executing it in the database
for query in housing_queries:
    # creating the cursor object
    cursor = connection.cursor()
    # executing the query
    cursor.execute(query)
    # closing the cursor object
    cursor.close()

# committing the queries to the database
connection.commit()

In [35]:
# displaying the newly created table for the query
display(pd.read_sql(""" 
SELECT *
FROM final_average_rent
;
""", engine).head())

,community_name,year,rental_rate_three_bedroom,assessed_value
0,Red Deer,2017,1084,12305180800
1,Red Deer,2018,1113,11978977005
2,Red Deer,2019,1139,11937517352
3,Red Deer,2020,1169,11644389137
4,Red Deer,2021,1163,11643244387


We have creating an interactive Tableau dashboard for this query, the link is:

https://public.tableau.com/app/profile/harjot.dhaliwal7759/viz/DATA604Housing_17015596693050/Dashboard1?publish=yes

We can see that the municipality with the highest average property assessment in 2021 is Calgary by a significant margin, followed by Edmonton, which is at about 2/3 of the value of Calgary, while the rest of the municipalities are clustered down at the bottom, significantly smaller than those 2. We can see for average rent rates, edmonton was the highest until about mid 2019, when Calgary overtook it and had the highest average rent rates, therefore it does appear there may be a relationship between the average rent rate and the average property assessment, as Calgary and Edmonton are significantly greater than the rest of the municipalities in 2021 in both metrics. The only exception to this rule would be the fact that in 2021 Red Deer had a higher average property assessment value than Lethbridge, but a lower average rent rate. This could show that Red Deer may be more desirable for renting, as opposed to purchasing a home. When choosing a major metropolitan area to live in Alberta it appears if you are looking to rent, a good option may be Medicine Hat or Red Deer, whereas if you are looking to buy a good place to live may be Medicine Hat or Red Deer. If you are unsure a good option would be Medicine Hat as it is the lowest for both of these factors.

Which municipality had the highest percentage of single family houses in 2021?

This can give us more insight into which municipalities are most likely to be good for families with children, as opposed to places which are more likely to be for young professionals, such as metropolitan areas.

In [37]:
# displaying the newly created table for the query
display(pd.read_sql(""" 
SELECT *
FROM final_perc_single_family
;
""", engine).head())

,community_name,year,dwelling_percentage
0,Norglenwold,2021,100
1,Arrowwood,2021,100
2,Jarvis Bay,2021,100
3,Rochon Sands,2021,100
4,Rosemary,2021,100


We have creating an interactive Tableau dashboard for this query, the link is:

https://public.tableau.com/app/profile/harjot.dhaliwal7759/viz/DATA604Housing2/Sheet1?publish=yes

It appears that most of the high percentage municipalities are more rural areas and the lower ones are larger municipalities such as Calgary and Edmonton. This would likely be due to the fact that larger municipalities have a higher proportion of condos and apartments, whereas people living in rural areas are more likely to be local people, who own a home and have been in Canada for a longer period of time. Also smaller towns would be more agriculture-focused, showing that there may be more of a need for single family homes in those areas, rather than professionals able to work from a computer anywhere. It is also interesting to see Canmore, Banff and Jasper being very low on this list, which is likely due to the fact that these locations are more for tourism and short visits, with the local population being very small, therefore there are likely to be more hotels and airbnbs in those municipalities.

### Business/Economy Queries

As local governments dispense funds for the construction of projects in their respective regions, there is a likely impact on the employment rates, income, the types of businesses, etc. The aim for these major projects is not only to improve the living conditions of the constituents but also make the economy more viable, which is what the investigation is primarily focused on. The query would be required to generate a Y-o-Y comparison for how the funding for different major projects have an impact on the business growth, economy and employment conditions.

In [39]:
business_queries = [
# dropping the table to create in case it exists to ensure code remains runnable
"""
DROP TABLE IF EXISTS final_business_growth;
""",
# creating the table for business growth
"""
CREATE TABLE final_business_growth AS
SELECT community_id, community_name, year, 
business_cnt - prev_business_cnt AS business_growth,
IF(prev_business_cnt > 0, ROUND((business_cnt - prev_business_cnt) / prev_business_cnt * 100, 2), 100) AS business_growth_percentage
FROM (
    SELECT a.*, COALESCE(LAG(a.business_cnt, 1) OVER (PARTITION BY a.community_id ORDER BY a.year), -1) AS prev_business_cnt
    FROM (
        # Aggregating the number of businesses for each year and municipality, the sum being a weighted sum based on the size of the business. 
        SELECT community_id, community_name, year, ROUND(SUM(
            CASE
            WHEN size = "Large" THEN num_business
            WHEN size = "Medium" THEN num_business * 0.75
            WHEN size = "Small" THEN num_business * 0.5
            END
        ), 0) AS business_cnt
        FROM int_number_of_businesses
        GROUP BY community_id, community_name, year
    ) a
) a WHERE prev_business_cnt <> -1;
""",
# dropping the table to create in case it exists to ensure code remains runnable
"""
DROP TABLE IF EXISTS final_project_funding;
""",
# creating the table for project funding
"""
CREATE TABLE final_project_funding AS
SELECT a.community_id, a.community_name, a.year,
ROUND((a.project_cost +
COALESCE(LAG(a.project_cost, 1) OVER (PARTITION BY a.community_id ORDER BY a.year), 0) +
COALESCE(LAG(a.project_cost, 2) OVER (PARTITION BY a.community_id ORDER BY a.year), 0) +
COALESCE(LAG(a.project_cost, 3) OVER (PARTITION BY a.community_id ORDER BY a.year), 0) +
COALESCE(LAG(a.project_cost, 4) OVER (PARTITION BY a.community_id ORDER BY a.year), 0)) / 5, 2) AS project_cost
FROM (
    SELECT community_id, community_name, year, 
    SUM(project_cost) AS project_cost FROM int_major_projects
    GROUP BY community_id, community_name, year
) a;
""",
# dropping the table to create in case it exists to ensure code remains runnable
"""
DROP TABLE IF EXISTS final_business_growth_and_project_funding;
""",
# creating the table for business growth and project funding
"""
CREATE TABLE final_business_growth_and_project_funding AS
SELECT a.community_id, a.community_name, a.year, 
a.business_growth, a.business_growth_percentage, COALESCE(b.project_cost, 0) AS project_cost 
FROM final_business_growth a
LEFT JOIN final_project_funding b ON a.community_id = b.community_id AND a.year = b.year
""",
# dropping the table to create in case it exists to ensure code remains runnable
"""
DROP TABLE IF EXISTS final_employment_income;
""",
# creating the final employment income table
"""
CREATE TABLE final_employment_income AS
SELECT community_id, community_name, year, employment_increase, employment_rate, income FROM (
    SELECT a.community_id, a.community_name, next_census_year AS year, a.employment_increase, a.employment_rate,
    ROUND(AVG(b.overall_income), 2) AS income FROM (
        SELECT community_id, community_name, year, next_census_year, overall_employment_rate AS employment_rate,
        CAST(LEAD(employed_count, 1) OVER (PARTITION BY community_id ORDER BY year) - employed_count AS INT) AS employment_increase
        FROM (
            SELECT a.community_id, a.community_name, a.year, b.overall_employment_rate,
            FLOOR(a.employable_total_count * b.overall_participation_rate * b.overall_employment_rate) as employed_count,
            FLOOR(a.employable_total_count * b.overall_participation_rate * b.overall_unemployment_rate) as unemployed_count,
            COALESCE(LEAD(a.year, 1) OVER (PARTITION BY a.community_id ORDER BY a.year), max_year.year) as next_census_year
            FROM int_population a
            INNER JOIN int_employment_census b ON a.community_id = b.community_id and a.year = b.year
            CROSS JOIN (SELECT MAX(year) AS year FROM int_population) max_year
        ) a
    ) a LEFT JOIN int_median_income b ON a.community_id = b.community_id AND b.year BETWEEN a.year AND a.next_census_year
    GROUP BY a.community_id, a.community_name, a.employment_increase
) a WHERE employment_increase IS NOT NULL;
""",
# dropping the table to create in case it exists to ensure code remains runnable
"""
DROP TABLE IF EXISTS final_employment_income_and_project_funding;
""",
# creating the final employment income and project funding table
"""
CREATE TABLE final_employment_income_and_project_funding AS
SELECT a.community_id, a.community_name, a.year, 
a.employment_increase, a.employment_rate, a.income, COALESCE(b.project_cost, 0) AS project_cost 
FROM final_employment_income a
LEFT JOIN final_project_funding b ON a.community_id = b.community_id AND a.year = b.year;
""",
# dropping the table to create in case it exists to ensure code remains runnable
"""
DROP TABLE IF EXISTS final_tax_incorporations;
""",
# creating the table for number of incorporations
"""
CREATE TABLE final_tax_incorporations AS
WITH JoinedData AS (
  SELECT 
    i.community_name,
    i.year,
    i.num_incorporations,
    t.tax_rate
  FROM 
    int_incorporations AS i
  INNER JOIN int_tax_rates AS t
    ON i.community_name = t.community_name AND i.year = t.year
),
Data2010 AS (
  SELECT 
    community_name,
    num_incorporations AS Inc2010,
    tax_rate AS Tax2010
  FROM JoinedData
  WHERE year = 2010
),
Data2020 AS (
  SELECT 
    community_name,
    num_incorporations AS Inc2020,
    tax_rate AS Tax2020
  FROM JoinedData
  WHERE year = 2020
)
SELECT 
  d2020.community_name AS community_name,
  (d2020.Inc2020 - d2010.Inc2010) / d2010.Inc2010 * 100 AS Incorporations_Change,
  (d2020.Tax2020 - d2010.Tax2010) / d2010.Tax2010 * 100 AS Tax_change
FROM 
  Data2020 d2020
INNER JOIN Data2010 d2010
  ON d2020.community_name = d2010.community_name;
  """]

# looping through each query above and executing it in the database
for query in business_queries:
    # creating the cursor object
    cursor = connection.cursor()
    # executing the query
    cursor.execute(query)
    # closing the cursor object
    cursor.close()

# committing the queries to the database
connection.commit()

In [40]:
# displaying the newly created tables for the querys
display(pd.read_sql(""" 
SELECT * FROM final_business_growth WHERE community_name = "Calgary"
ORDER BY year DESC LIMIT 5
""", engine))

display(pd.read_sql(""" 
SELECT * FROM final_project_funding WHERE community_name = "Calgary"
ORDER BY year DESC LIMIT 5
""", engine))

display(pd.read_sql(""" 
SELECT * FROM final_employment_income WHERE community_name = "Calgary"
ORDER BY year DESC LIMIT 5
""", engine))

display(pd.read_sql(""" 
SELECT * FROM final_business_growth_and_project_funding WHERE community_name = "Calgary"
ORDER BY year DESC LIMIT 5
""", engine))

display(pd.read_sql(""" 
SELECT * FROM final_employment_income_and_project_funding WHERE community_name = "Calgary"
ORDER BY year DESC LIMIT 5
""", engine))

,community_id,community_name,year,business_growth,business_growth_percentage
0,4806016,Calgary,2022,571,2
1,4806016,Calgary,2021,-236,-1
2,4806016,Calgary,2020,7,0
3,4806016,Calgary,2019,53,0
4,4806016,Calgary,2018,291,1


,community_id,community_name,year,project_cost
0,4806016,Calgary,2022,16367
1,4806016,Calgary,2021,16426
2,4806016,Calgary,2020,17078
3,4806016,Calgary,2019,17462
4,4806016,Calgary,2018,18022


,community_id,community_name,year,employment_increase,employment_rate,income
0,4806016,Calgary,2021,-32321,1,104914
1,4806016,Calgary,2016,21305,1,98868
2,4806016,Calgary,2011,25102,1,88063
3,4806016,Calgary,2006,70604,1,70983


,community_id,community_name,year,business_growth,business_growth_percentage,project_cost
0,4806016,Calgary,2022,571,2,16367
1,4806016,Calgary,2021,-236,-1,16426
2,4806016,Calgary,2020,7,0,17078
3,4806016,Calgary,2019,53,0,17462
4,4806016,Calgary,2018,291,1,18022


,community_id,community_name,year,employment_increase,employment_rate,income,project_cost
0,4806016,Calgary,2021,-32321,1,104914,16426
1,4806016,Calgary,2016,21305,1,98868,18725
2,4806016,Calgary,2011,25102,1,88063,25541
3,4806016,Calgary,2006,70604,1,70983,110


We have creating an interactive Tableau dashboard for this query, the link is:

https://public.tableau.com/app/profile/shabbir.khandwala6032/viz/BusinessGrowthFundingv2/Dashboard1?publish=yes

While government funding doesn't guarantee business growth, it is infact necessary for atleast pushing for more businesses within the locality. There is a consistent increase in the number of businesses as the funding consistently gets poured in with a few erratic drops. This suggests that government funding is definitely important for growing the number of businesses. Secondly, the employment rate of the eligible population remains fairly healthy with increases in the median income of the municipality particularly as more and more projects get funded. Both are these are relatively higher for areas where there is significant amount of dollars being poured into various projects.

How do Taxes affect business creation?

It's important to connect taxes and the creation of new businesses because these factors are closely linked. A higher tax rate means higher costs for new businesses, which is bad for them, while a lower tax rate is good because it means lower costs for new businesses in the future.

In [43]:
# displaying the newly created table for the query
display(pd.read_sql(""" 
SELECT *
FROM final_tax_incorporations
;
""", engine).head())

,community_name,Incorporations_Change,Tax_change
0,Sedgewick,0,-14
1,Lougheed,0,-15
2,Red Deer,-11,13
3,Strathmore,-27,38
4,Standard,300,30


We have creating an interactive Tableau dashboard for this query, the link is:

https://public.tableau.com/app/profile/shabbir.khandwala6032/viz/TaxChangeBusinessChange/Final?publish=yes

When looking at the graph, we can see that most of the points are found when the % change in tax is positive. This tells us that increase in tax rates do not necessarily decrease the number of business incorporations in a municipality as one would believe.

Closing connection to database for SQLAlchemy and MySQL Python connector:

In [42]:
connection.close()
engine.dispose()

# **Discussion**

Our project took a close look at three important areas in Alberta: housing, health, and business. We discovered interesting details that help us understand the region better. In the Health and Environment category, we found that air quality is consistently good, especially in rural areas. The population is growing slowly or, in some cases, decreasing a bit, but cities like Calgary and Edmonton lead in births. Turning to housing, we noticed a strong connection between property values and rent rates, giving us a clue about future housing trends. In the Business and Economy field, government projects play a big role in business growth, especially in Calgary and Edmonton. Surprisingly, higher taxes don't always mean fewer businesses, challenging what we might expect.

Now, let's think about what we've learned and how we can do better in future projects. This experience really helped us improve our skills in using SQL. We focused on making our queries run faster, showing our commitment to doing great work. Using Tableau to visualize data became one of our strong points, making complex ideas easy to understand. Looking ahead, we're excited to learn more, especially about other database systems like MongoDB and Redis. Real-world situations are the best way to use what we know and learn even more. We're also interested in trying out other tools like Power BI to make our projects even better. Having a variety of skills makes us ready for anything in the world of data analysis.

In a nutshell, this project was more than just looking at Alberta's sectors. It helped us grow personally and as a team, improving our skills, questioning what we thought we knew, and making us eager to learn more. As we think about what's next, we're ready to use our skills and insights to achieve even more in the world of data analysis and exploration.

# **Conclusion**

During this project, we carefully studied data from Alberta and its various municipalities. We worked with relational CSV datasets, looking at important factors in Housing, Health/Environment, and Business/Economy. Our main goal was to learn how to use advanced tools like SQL and Tableau to organize and query the data, so we could find answers to our research questions. We also focused on creating clear and varied visualizations to show how different areas in Alberta perform across chosen metrics. Our findings are useful for both long-time residents and those thinking about moving to Alberta. For residents, it's a helpful resource to find areas for improvement or where the province might need to step in. Prospective residents can use this info to understand which places might be a good fit for them. In a nutshell, our project gives a modern view of Alberta's recent years, highlighting areas that could use attention or improvement. To sum up our learning journey, our skills in SQL, along with tools like SQL Workbench and Python connectors, helped us make sense of the data. Creating interactive Tableau dashboards showed our commitment to making the data easy to explore.

In the end, our mission was successful. We didn't just dive into the details of municipality data – we went the extra mile to present it in a way that's easy to understand. Our project shows our dedication to giving valuable information to Alberta residents and potential newcomers, so they can make well-informed decisions in the ever-changing province.

# **References**

Alberta NDP calls on Province to Fully Commit to Calgary's Green Line Funding. (2023).CTV News. https://calgary.ctvnews.ca/alberta-ndp-calls-on-province-to-fully-commit-to-calgary-s-green-line-funding-1.6479706

Air Quality Index by Municipality. (2023). Government of Alberta. https://www.alberta.ca/open-government-program

Average Rent by Municipality. (2023). Government of Alberta. https://www.alberta.ca/open-government-program/average-rents-by-municipality

Births and Deaths by Municipality. (2023). Government of Alberta. https://www.alberta.ca/open-government-program

Businesses by Municipality. (2023). Government of Alberta. https://open.alberta.ca/opendata/businesses-by-municipality 

Calgary City Council to Hold Special Meeting to Address ‘Housing Crisis'. (2023). CBC News. https://www.cbc.ca/news/canada/calgary/housing-needs-assessment-report-calgary-housing-crisis-1.6958518

Census employment by Municipality. (2023). Government of Alberta. https://open.alberta.ca/opendata/census-employment-by-municipality

Incorporations by Municipality. (2023). Government of Alberta. https://open.alberta.ca/opendata/incorporations-by-municipality

Major projects by Municipality. (2023). Government of Alberta. https://www.alberta.ca/open-government-program/major-projects-by-municipality

Median income by Municipality. (2023). Government of Alberta. https://open.alberta.ca/opendata/median-income-by-municipality

Municipal Tax Rates by Municipality. (2023). Government of Alberta. https://open.alberta.ca/opendata/municipal-tax-rate-by-municipality

Open Government Program. (2023). Government of Alberta. https://www.alberta.ca/open-government-program

Percent Single Family Houses by Municipality. (2023). Government of Alberta. https://www.alberta.ca/open-government-program/percent-single-family-houses-by-municipality

Property Assessments by Municipality. (2023). Government of Alberta. https://www.alberta.ca/open-government-program/total-equalized-assessment-by-municipality

Population by Municipality. (2023). Government of Alberta. https://www.alberta.ca/open-government-program

Wildfires in Alberta Could Burn All Winter: Official. (2023). Global News. https://globalnews.ca/news/10014200/alberta-wildfires-burning-season-ending/